In [1]:
from dotenv import load_dotenv
load_dotenv()
import wandb


R = 8
LR= 2e-4,
EPOCHS=10
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="tool-learning",
    name="sample",

    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "r": R,
    "dataset": "api_bank",
    "epochs": EPOCHS,
    }
)



KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL = "microsoft/phi-2"# "microsoft/phi-2"
# bnb_config = BitsAndBytesConfig(
#     # load_in_4bit=True,
#     # bnb_4bit_quant_type="nf4",
#     # bnb_4bit_compute_dtype=torch.bfloat16
# )

tokeniser = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL, 
                                                #  attn_implementation="flash_attention_2",
                                                #  quantization_config = ...
                                             )#quantization_config=bnb_config


In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=R,
    # target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
import pandas as pd


api_bank_df = pd.read_csv('Datasets/API-Bank/cleaned_api_bank_data.csv')

print(api_bank_df.prompt.iloc[0])
print(api_bank_df.completion.iloc[0])


Generate an API request in the format of [ApiName(key1='value1', key2='value2', ...)] based on the previous dialogue context.
The current time is 2039-03-09 18:56:09 Wednesday.
Input: 
User: User's utterence
AI: AI's response

Expected output:
API-Request: [ApiName(key1='value1', key2='value2', ...)]

API descriptions:

{"apiCode": "Get_All_Sessions", "description": "Get the list of all available yoga and meditation sessions.", "parameters": {}, "response": {"data": {"description": "List of available sessions.", "type": "list", "items": {"type": "object", "properties": {"session_name": {"description": "Name of the session.", "type": "string"}, "session_date": {"description": "Date of the session.", "type": "string", "format": "formatted"}, "session_time": {"description": "Time of the session.", "type": "string", "format": "formatted"}, "session_instructor": {"description": "Name of the session instructor.", "type": "string"}, "session_description": {"description": "Description of the 

In [ ]:
from datasets import Dataset 
api_train = Dataset.from_pandas(api_bank_df[api_bank_df['split']=='train'])
api_test = Dataset.from_pandas(api_bank_df[api_bank_df['split']=='test'])
api_eval = Dataset.from_pandas(api_bank_df[api_bank_df['split']=='eval'])

for dataset in [api_train, api_test, api_eval]:
    print(dataset)

Dataset({
    features: ['Unnamed: 0', 'instruction', 'input', 'output', 'split', 'query_type', 'completion', 'instruction_query_type', 'function_calls', 'number_of_calls', 'function_call', 'function_name', 'prompt', '__index_level_0__'],
    num_rows: 9321
})
Dataset({
    features: ['Unnamed: 0', 'instruction', 'input', 'output', 'split', 'query_type', 'completion', 'instruction_query_type', 'function_calls', 'number_of_calls', 'function_call', 'function_name', 'prompt', '__index_level_0__'],
    num_rows: 997
})
Dataset({
    features: ['Unnamed: 0', 'instruction', 'input', 'output', 'split', 'query_type', 'completion', 'instruction_query_type', 'function_calls', 'number_of_calls', 'function_call', 'function_name', 'prompt', '__index_level_0__'],
    num_rows: 1047
})


In [ ]:
# https://huggingface.co/docs/trl/en/sft_trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments 

training_args = TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            max_snumteps=EPOCHS,
            learning_rate=LR,
            fp16=False,
            logging_steps=1,
            output_dir="outputs",
            optim="paged_adamw_8bit",
            report_to="wandb",
        )


def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Prompt: {example['prompt'][i]}\n ### Completion: {example['completion'][i]}"
        output_texts.append(text)
    return output_texts


trainer = SFTTrainer(
    model=model,
    train_dataset=api_train,
    eval_dataset=api_eval,
        args=training_args,
    peft_config=lora_config,
    # data_collator=data_collator
    formatting_func = formatting_prompts_func
)
# trainer.train()
trainer





max_steps is given, it will override any value given in num_train_epochs
